In [ ]:
import os
import win32file
import msvcrt

import pymedphys

In [ ]:
data_paths = pymedphys.zip_data_paths("monaco_to_trf.zip")

In [ ]:
tel_paths = [path for path in data_paths if "tel" in path.name]
assert len(tel_paths) == 1
tel_path = tel_paths[0]

In [ ]:
tel_path

In [ ]:
handle = win32file.CreateFile(
    str(tel_path),
    win32file.GENERIC_READ,
    win32file.FILE_SHARE_DELETE |
    win32file.FILE_SHARE_READ |
    win32file.FILE_SHARE_WRITE,
    None,
    win32file.OPEN_EXISTING,
    0,
    None)


detached_handle = handle.Detach()

file_descriptor = msvcrt.open_osfhandle(
    detached_handle, os.O_RDONLY)

with open(file_descriptor) as file:
    data = file.read()

In [ ]:
data